In [ ]:
# set up
import numpy as np
import os
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, Activation
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras import optimizers
from keras import regularizers
from keras.utils import to_categorical
import gc

In [ ]:
# load data
num_frames = 15
npy_path = os.getcwd() + '/datasets/'
train = np.load(npy_path + 'cache/train_num_videos4270_num_frame15.npz')
Xtr = train['Xtrain']
ytr = train['ytrain']
print('Xtrain shape: ', Xtr.shape)
print('ytrain shape: ', ytr.shape)

In [ ]:
# extract sample images per video
sample = 3
Xtr_re = Xtr.reshape((-1, 15, 7, 7, 512))[:, :sample, :, :, :]
Xtr_re = Xtr_re.reshape((-1, 7, 7, 512))
print('reshape xtrain: ', Xtr_re.shape)

In [ ]:
ytr_re = ytr.reshape((-1, 1))
ytr_re = np.repeat(ytr_re, sample, axis = 1).reshape((Xtr_re.shape[0],))
print('reshape ytrain: ', ytr_re.shape)

In [ ]:
del Xtr
del ytr
gc.collect()

In [ ]:
def classifier(optimizer, reg = 0.01):
    x = Input(shape = (7, 7, 512))
    x_flat = Flatten()(x)
    fc1 = Dense(units = 4096, kernel_regularizer=regularizers.l2(reg))(x_flat)
    fc2 = Dense(units = 256, kernel_regularizer=regularizers.l2(reg))(fc1)
    fc3 = Dense(units = 10, kernel_regularizer=regularizers.l2(reg))(fc2)
    sf = Activation('softmax')(fc3)
    model = Model(inputs=x, outputs=sf)
    model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    return model

In [ ]:
def train(Xtr, ytr, model, epochs, bsize, split_ratio = 0.2, verbose = False):
    
    print('Model is Training...')
    checkpointer = ModelCheckpoint(filepath='frame_classification_weights.hdf5', verbose=1, save_best_only=True)

    # reduce learning rate when on plateau
    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
    
    hist = model.fit(Xtr, ytr, epochs=epochs, batch_size= bsize, validation_split = split_ratio, \
                             verbose = verbose, callbacks=[checkpointer, reduceLR])
    return hist

In [ ]:
# training
adam = optimizers.Adam(lr=3e-6, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-6)
cf = classifier(optimizer = adam, reg = 0.01)
ytr_re = to_categorical(ytr_re, num_classes = 10)
hist = train(Xtr_re, ytr_re, cf, epochs = 10, bsize = 64, verbose = True)

In [ ]:
# clear model
cf = None
gc.collect()